In [1]:
import contextlib
import glob
import os

import pytorch_lightning as pl
import torch

from nemo.collections.asr.metrics.rnnt_wer import RNNTDecodingConfig
from nemo.core.config import hydra_runner
from nemo.collections.asr.models import EncDecCTCModelBPE

[NeMo W 2021-06-04 16:04:40 optimizers:46] Apex was not found. Using the lamb optimizer will error out.
[NeMo W 2021-06-04 16:04:42 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [2]:
audio_dir = '/home/dino/datasets/ICSI/Signals_WAV/Bmr028/'  # Path to a directory which contains audio files

# General configs
output_filename: str = "speech_to_text_transcriptions.txt"
amp: bool = False
audio_type: str = "wav"

# decoding strategy for RNNT models
rnnt_decoding: RNNTDecodingConfig = RNNTDecodingConfig()

cuda = torch.cuda.is_available()

In [3]:
cuda

True

In [4]:
asr_model = EncDecCTCModelBPE.from_pretrained(
    model_name='stt_en_conformer_ctc_small', map_location='cuda:0'
)

[NeMo I 2021-06-04 16:04:42 cloud:56] Found existing object /home/dino/.cache/torch/NeMo/NeMo_1.0.0/stt_en_conformer_ctc_small/9d26c5e7c094a612f32b67a4c35f2055/stt_en_conformer_ctc_small.nemo.
[NeMo I 2021-06-04 16:04:42 cloud:62] Re-using file from: /home/dino/.cache/torch/NeMo/NeMo_1.0.0/stt_en_conformer_ctc_small/9d26c5e7c094a612f32b67a4c35f2055/stt_en_conformer_ctc_small.nemo
[NeMo I 2021-06-04 16:04:42 common:675] Instantiating model from pre-trained checkpoint


Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


[NeMo I 2021-06-04 16:04:46 mixins:147] Tokenizer AutoTokenizer initialized with 128 tokens


[NeMo W 2021-06-04 16:04:47 modelPT:138] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/librispeech_withsp2/manifests/librispeech_train_withSP.json
    sample_rate: 16000
    batch_size: 128
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: false
    tarred_audio_filepaths: /data/asr_set_1.4/train_no_appen/train_no_appen__OP_0..1023_CL_.tar
    
[NeMo W 2021-06-04 16:04:47 modelPT:145] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/librispeech_withsp2/manifests/librivox-dev-other.json
    - /data/librispeech_withsp2/manifests/librivo

[NeMo I 2021-06-04 16:04:47 features:252] PADDING: 0
[NeMo I 2021-06-04 16:04:47 features:269] STFT using torch
[NeMo I 2021-06-04 16:04:49 modelPT:439] Model EncDecCTCModelBPE was successfully restored from /home/dino/.cache/torch/NeMo/NeMo_1.0.0/stt_en_conformer_ctc_small/9d26c5e7c094a612f32b67a4c35f2055/stt_en_conformer_ctc_small.nemo.


In [5]:
trainer = pl.Trainer(gpus=int(cuda))
asr_model.set_trainer(trainer)
asr_model = asr_model.eval()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [6]:
# Setup decoding strategy
if hasattr(asr_model, 'change_decoding_strategy'):
    asr_model.change_decoding_strategy(rnnt_decoding)

# load paths to audio
filepaths = list(glob.glob(os.path.join(audio_dir, f"*.{audio_type}")))

@contextlib.contextmanager
def autocast():
    yield

In [7]:
# transcribe audio
with autocast():
    with torch.no_grad():
        transcriptions = asr_model.transcribe(filepaths, batch_size=1)

Transcribing:   0%|          | 0/15 [00:00<?, ?it/s]

[NeMo W 2021-06-04 16:07:01 patch_utils:49] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.


RuntimeError: CUDA out of memory. Tried to allocate 4.34 GiB (GPU 0; 7.79 GiB total capacity; 4.70 GiB already allocated; 1.35 GiB free; 4.81 GiB reserved in total by PyTorch)

In [ ]:
logging.info(f"Writing transcriptions into file: {output_filename}")
with open(output_filename, 'w', encoding='utf-8') as f:
    for line in transcriptions:
        f.write(f"{line}\n")